<a href="https://colab.research.google.com/github/aguscura/UBA-data-science/blob/main/spark_S5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**S5:** Para los negocios que indican su horario, calcule la probabilidad de que un negocio esté abierto a las 21 horas del domingo. Si el domingo no apareciera en el diccionario asuma que no abre.(1 PUNTO)

In [2]:
!pip install pyspark
# !pip install -U -q PyDrive
# !apt update
# !apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 199 kB 45.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=040c13f5b33e1e94adf1331cd8502a58c39bba209196ecfe70335ca7a248734e
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [51]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
from tqdm import tqdm
import json

In [4]:
!gdown "14sjAJe8wBi7N_1xxJFCm7Mtzrm_gFiPj"

Downloading...
From: https://drive.google.com/uc?id=14sjAJe8wBi7N_1xxJFCm7Mtzrm_gFiPj
To: /content/business.csv
100% 99.3M/99.3M [00:00<00:00, 124MB/s]


In [6]:
#Chequeo columnas del df
df = pd.read_csv("/content/business.csv")
df.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,0.0,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,8.0,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,11.0,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,37.0,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13.0,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [8]:
#Creamos el Spark Context
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Paso el csv a parquet. Por un tema de RAM solo me traigo algunas columnas

In [7]:
#Paso a parquet. Acá solo tomo algunas columnas por el RAM
df = pd.read_csv("/content/business.csv", usecols=["business_id","name","is_open", "hours"])
df.to_parquet("/content/business.parquet")

In [9]:
#Leemos archivo parquet
sqlContext = SQLContext(sc)
rdd = sqlContext.read.parquet('/content/business.parquet').rdd.repartition(2)


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


"para los negocios que indican su horario"... hay que filtrar.

**EL ANALISIS SOLO CONTEMPLA LOS NEGOCIOS QUE INDICAN EL HORARIO.**

In [36]:
#Todos los registros
rdd.count()

150346

In [37]:
#Registros nulos
rdd.map(lambda x: x.hours).filter(pd.isnull).count()

23223

In [143]:
#Filtro los negocios que no indican su horario y los cuento - nulos + no nulos = totales (check)
totales_con_horario = rdd.map(lambda x: x.hours).filter(pd.notnull).count()
totales_con_horario

127123

In [53]:
rdd.take(5)

[Row(business_id='Pns2l4eNsfO8kk83dixA6A', name='Abby Rappoport, LAC, CMQ', is_open=0, hours=None),
 Row(business_id='mpf3x-BjTdTEA3yCZrAYPw', name='The UPS Store', is_open=1, hours="{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', 'Wednesday': '8:0-18:30', 'Thursday': '8:0-18:30', 'Friday': '8:0-18:30', 'Saturday': '8:0-14:0'}"),
 Row(business_id='tUFrWirKiKi_TAnsVWINQQ', name='Target', is_open=0, hours="{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', 'Wednesday': '8:0-22:0', 'Thursday': '8:0-22:0', 'Friday': '8:0-23:0', 'Saturday': '8:0-23:0', 'Sunday': '8:0-22:0'}"),
 Row(business_id='MTSW4McQd7CbVtyjqoe9mw', name='St Honore Pastries', is_open=1, hours="{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', 'Wednesday': '7:0-20:0', 'Thursday': '7:0-20:0', 'Friday': '7:0-21:0', 'Saturday': '7:0-21:0', 'Sunday': '7:0-21:0'}"),
 Row(business_id='mWMc6_wTdE0EUBKIGXDVfA', name='Perkiomen Valley Brewery', is_open=1, hours="{'Wednesday': '14:0-22:0', 'Thursday': '16:0-22:0', 'Friday': '12:0-22:0', 'Satu

In [56]:
rdd.map(lambda x: x.hours).filter(pd.notnull).take(5)

["{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', 'Wednesday': '8:0-18:30', 'Thursday': '8:0-18:30', 'Friday': '8:0-18:30', 'Saturday': '8:0-14:0'}",
 "{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', 'Wednesday': '8:0-22:0', 'Thursday': '8:0-22:0', 'Friday': '8:0-23:0', 'Saturday': '8:0-23:0', 'Sunday': '8:0-22:0'}",
 "{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', 'Wednesday': '7:0-20:0', 'Thursday': '7:0-20:0', 'Friday': '7:0-21:0', 'Saturday': '7:0-21:0', 'Sunday': '7:0-21:0'}",
 "{'Wednesday': '14:0-22:0', 'Thursday': '16:0-22:0', 'Friday': '12:0-22:0', 'Saturday': '12:0-22:0', 'Sunday': '12:0-18:0'}",
 "{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', 'Wednesday': '6:0-22:0', 'Thursday': '6:0-22:0', 'Friday': '9:0-0:0', 'Saturday': '9:0-22:0', 'Sunday': '8:0-22:0'}"]

In [70]:
hours_cache = rdd.map(lambda x: x.hours).filter(pd.notnull).cache()
hours_cache.take(5)

["{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', 'Wednesday': '8:0-18:30', 'Thursday': '8:0-18:30', 'Friday': '8:0-18:30', 'Saturday': '8:0-14:0'}",
 "{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', 'Wednesday': '8:0-22:0', 'Thursday': '8:0-22:0', 'Friday': '8:0-23:0', 'Saturday': '8:0-23:0', 'Sunday': '8:0-22:0'}",
 "{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', 'Wednesday': '7:0-20:0', 'Thursday': '7:0-20:0', 'Friday': '7:0-21:0', 'Saturday': '7:0-21:0', 'Sunday': '7:0-21:0'}",
 "{'Wednesday': '14:0-22:0', 'Thursday': '16:0-22:0', 'Friday': '12:0-22:0', 'Saturday': '12:0-22:0', 'Sunday': '12:0-18:0'}",
 "{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', 'Wednesday': '6:0-22:0', 'Thursday': '6:0-22:0', 'Friday': '9:0-0:0', 'Saturday': '9:0-22:0', 'Sunday': '8:0-22:0'}"]

In [122]:
dict_hours = hours_cache.map(lambda x: json.loads(x.replace("\'", "\"")))
dict_hours.take(2)

[{'Monday': '0:0-0:0',
  'Tuesday': '8:0-18:30',
  'Wednesday': '8:0-18:30',
  'Thursday': '8:0-18:30',
  'Friday': '8:0-18:30',
  'Saturday': '8:0-14:0'},
 {'Monday': '8:0-22:0',
  'Tuesday': '8:0-22:0',
  'Wednesday': '8:0-22:0',
  'Thursday': '8:0-22:0',
  'Friday': '8:0-23:0',
  'Saturday': '8:0-23:0',
  'Sunday': '8:0-22:0'}]

In [138]:
sunday_hours = dict_hours.map( lambda x: x["Sunday"].split("-") if "Sunday" in x else ["0:0","0:0"])
sunday_hours.take(20)

[['0:0', '0:0'],
 ['8:0', '22:0'],
 ['7:0', '21:0'],
 ['12:0', '18:0'],
 ['8:0', '22:0'],
 ['12:0', '18:0'],
 ['0:0', '0:0'],
 ['6:0', '21:0'],
 ['10:0', '17:0'],
 ['11:0', '19:0'],
 ['0:0', '0:0'],
 ['11:0', '21:0'],
 ['0:0', '0:0'],
 ['11:0', '18:0'],
 ['0:0', '0:0'],
 ['11:0', '0:0'],
 ['10:0', '0:0'],
 ['9:0', '15:0'],
 ['16:0', '21:30'],
 ['0:0', '0:0']]

In [132]:
def time_to_num(time_str):
    hh, mm = map(int, time_str.split(':'))
    return ((mm/60) + hh)

In [139]:
hours = sunday_hours.map(lambda x: [time_to_num(x[0]), time_to_num(x[1])]  )
hours.take(20)

[[0.0, 0.0],
 [8.0, 22.0],
 [7.0, 21.0],
 [12.0, 18.0],
 [8.0, 22.0],
 [12.0, 18.0],
 [0.0, 0.0],
 [6.0, 21.0],
 [10.0, 17.0],
 [11.0, 19.0],
 [0.0, 0.0],
 [11.0, 21.0],
 [0.0, 0.0],
 [11.0, 18.0],
 [0.0, 0.0],
 [11.0, 0.0],
 [10.0, 0.0],
 [9.0, 15.0],
 [16.0, 21.5],
 [0.0, 0.0]]

In [142]:
#Considero que a las 21 el negocio cierra, por lo que NO está abierto.
abiertos = hours.map(lambda x: 1 if ( (21 > float(x[0])) and (21 < float(x[1]) ) ) else 0).reduce(lambda a,b: a+b)
abiertos

15479

In [144]:
(abiertos/totales_con_horario) *100

12.176396088827355

15479 locales de los 127123 locales que poseen horario están abiertos los domingos a las 21hs.

**12.176%**